## **Nivel PySpark**

**Instalação do PySpark**

In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 51.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=c2493c164d9eed9ced13b7a6495e6287580f76f1e62f4d2b6b139372cf806345
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
pip install gcsfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 7.6 MB/s 
     |████████████████████████████████| 140 kB 65.3 MB/s 
     |████████████████████████████████| 144 kB 60.3 MB/s 
     |████████████████████████████████| 94 kB 2.5 MB/s 
     |████████████████████████████████| 271 kB 48.8 MB/s 


**Importar bibliotecas**

In [85]:
# CHAMAR AS BIBLIOTECAS e MÓDULOS NECESSÁRIAS
import os
import csv
import pyspark.sql.functions as F
from pyspark import SparkConf
from google.colab import drive
from google.cloud import storage
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType, FloatType, DoubleType

In [8]:
drive.mount('/content/drive')

Mounted at /content/drive


### **Configuração de Acesso para os GCP**

In [9]:
# CONFIGURAÇÃO DA CHAVE DE SEGURANÇA

serviceAccount = '/content/weighty-archive-349117-ab0ee1ad5b41.json'

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount


In [13]:
# CÓDIGO QUE ACESSA A BUCKET CRIADA E FAZ O DOWNLOAD DOS ARQUIVOS VIA PANDAS

client = storage.Client()

# CRIAR UMA VARIÁVEL CHAMADA BUCKET QUE VAI RECEBER O NOME DA BUCKET DO CLOUD STORAGE
bucket = client.get_bucket('soulcodemarketing')

# USAR O MÉTODO BLOB PARA RETORNAR O NOME DO ARQUIVO (JSON, CSV, PARQUET)
bucket.blob('marketing_campaign.csv')

 #CRIA UMA VARIÁVEL PATH PARA COLOCAR O CAMINHO DO CSV
path = 'gs://soulcodemarketing/marketing_campaign.csv'


In [14]:
# CRIAR A SPARK SESSION E LER O ARQUIVO VIA PYSPARK

spark = (SparkSession.builder\
        .master("local")\
        .appName("mkt_dataframe")\
        .config("spark.ui.port", "4050")\
        .config("spark.jars", 'https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop2-latest.jar')\
        .getOrCreate())

In [15]:
spark

In [16]:
# CRIAR UM DATAFRAME PELO PYSPARK

df = ( spark.read.format('csv')
                 .option('delimiter', ';')
                 .option('header', 'true')
                 .option('inferschema', 'true')
                 .load('gs://soulcodemarketing/marketing_campaign.csv')

)

In [17]:
df.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Year_Birth: integer (nullable = true)
 |-- Education: string (nullable = true)
 |-- Marital_Status: string (nullable = true)
 |-- Income: integer (nullable = true)
 |-- Kidhome: integer (nullable = true)
 |-- Teenhome: integer (nullable = true)
 |-- Dt_Customer: string (nullable = true)
 |-- Recency: integer (nullable = true)
 |-- MntWines: integer (nullable = true)
 |-- MntFruits: integer (nullable = true)
 |-- MntMeatProducts: integer (nullable = true)
 |-- MntFishProducts: integer (nullable = true)
 |-- MntSweetProducts: integer (nullable = true)
 |-- MntGoldProds: integer (nullable = true)
 |-- NumDealsPurchases: integer (nullable = true)
 |-- NumWebPurchases: integer (nullable = true)
 |-- NumCatalogPurchases: integer (nullable = true)
 |-- NumStorePurchases: integer (nullable = true)
 |-- NumWebVisitsMonth: integer (nullable = true)
 |-- AcceptedCmp3: integer (nullable = true)
 |-- AcceptedCmp4: integer (nullable = true)
 |-- AcceptedC

In [18]:
df.show()

+----+----------+----------+--------------+------+-------+--------+-----------+-------+--------+---------+---------------+---------------+----------------+------------+-----------------+---------------+-------------------+-----------------+-----------------+------------+------------+------------+------------+------------+--------+-------------+---------+--------+
|  ID|Year_Birth| Education|Marital_Status|Income|Kidhome|Teenhome|Dt_Customer|Recency|MntWines|MntFruits|MntMeatProducts|MntFishProducts|MntSweetProducts|MntGoldProds|NumDealsPurchases|NumWebPurchases|NumCatalogPurchases|NumStorePurchases|NumWebVisitsMonth|AcceptedCmp3|AcceptedCmp4|AcceptedCmp5|AcceptedCmp1|AcceptedCmp2|Complain|Z_CostContact|Z_Revenue|Response|
+----+----------+----------+--------------+------+-------+--------+-----------+-------+--------+---------+---------------+---------------+----------------+------------+-----------------+---------------+-------------------+-----------------+-----------------+----------

### **1 - Deverá ser montada a estrutura do DataFrame utilizando o StructType.**

**StructType**

In [40]:
schema = StructType([ 
      StructField("",                       IntegerType(), True),
      StructField("id",                     IntegerType(), True),
      StructField("ano_nascimento",         IntegerType(),  True),
      StructField("educacao",               StringType(),  True),
      StructField("estado_civil",           StringType(),   True),
      StructField("renda_anual",            FloatType(),   True),
      StructField("qtd_crianca",            IntegerType(),  True),
      StructField("qtd_adolescente",        IntegerType(), True),
      StructField("primeira_data_compra",   DateType(),    True),
      StructField("dias_desde_ult_compras", IntegerType(), True),
      StructField("qtd_vinhos",             IntegerType(), True),
      StructField("qtd_frutas",             IntegerType(), True),
      StructField("qtd_carne",              IntegerType(), True),
      StructField("qtd_peixe",              IntegerType(), True),
      StructField("guloseimas",             IntegerType(), True),
      StructField("qtd_produtos_gold",      IntegerType(), True),
      StructField("qtd_ofertas_compra",     IntegerType(), True),
      StructField("qtd_compras_online",     IntegerType(), True),
      StructField("qtd_compras_catalogo",   IntegerType(), True),
      StructField("qtd_compras_na_loja",    IntegerType(), True),
      StructField("qtd_visitas_online_mes", IntegerType(), True),
      StructField("oferta_aceita1",         StringType(),  True),
      StructField("oferta_aceita2",         StringType(),  True),
      StructField("oferta_aceita3",         StringType(),  True),
      StructField("oferta_aceita4",         StringType(),  True),
      StructField("oferta_aceita5",         StringType(),  True),
      StructField("reclamacao",             StringType(),  True)])

In [45]:
local= ("/content/df_pd_marketing_campaign.csv")

In [52]:
# Leitura do dataframe
df = spark.read.load(local, format="csv", header="true", sep=',', schema=schema)

In [53]:
df.show()  # Visualização de dataframe

+---+----+--------------+------------------+-------------+-----------+-----------+---------------+--------------------+----------------------+----------+----------+---------+---------+----------+-----------------+------------------+------------------+--------------------+-------------------+----------------------+--------------+--------------+--------------+--------------+--------------+----------+
|   |  id|ano_nascimento|          educacao| estado_civil|renda_anual|qtd_crianca|qtd_adolescente|primeira_data_compra|dias_desde_ult_compras|qtd_vinhos|qtd_frutas|qtd_carne|qtd_peixe|guloseimas|qtd_produtos_gold|qtd_ofertas_compra|qtd_compras_online|qtd_compras_catalogo|qtd_compras_na_loja|qtd_visitas_online_mes|oferta_aceita1|oferta_aceita2|oferta_aceita3|oferta_aceita4|oferta_aceita5|reclamacao|
+---+----+--------------+------------------+-------------+-----------+-----------+---------------+--------------------+----------------------+----------+----------+---------+---------+----------+-

### **2 - Verificar a existência de dados inconsistentes, nulos e realizar a limpeza.**

Os dados utilizados formam tratados na etapa 1 - pandas.

### **3 - Verificar a existência de dados inconsistentes, nulos e realizar a limpeza.**

Os dados formam tratados na etapa 1 - pandas.

### **4 - Realizar a mudança de nome de pelo menos 2 colunas**

In [57]:
# Modifica o id e a primeira_data_compra
df=df.withColumnRenamed("id", "identificacao").withColumnRenamed("primeira_data_compra", "data_primeira_compra")

In [58]:
df.show()  # visualizar dataframe

+---+-------------+--------------+------------------+-------------+-----------+-----------+---------------+--------------------+----------------------+----------+----------+---------+---------+----------+-----------------+------------------+------------------+--------------------+-------------------+----------------------+--------------+--------------+--------------+--------------+--------------+----------+
|   |identificacao|ano_nascimento|          educacao| estado_civil|renda_anual|qtd_crianca|qtd_adolescente|data_primeira_compra|dias_desde_ult_compras|qtd_vinhos|qtd_frutas|qtd_carne|qtd_peixe|guloseimas|qtd_produtos_gold|qtd_ofertas_compra|qtd_compras_online|qtd_compras_catalogo|qtd_compras_na_loja|qtd_visitas_online_mes|oferta_aceita1|oferta_aceita2|oferta_aceita3|oferta_aceita4|oferta_aceita5|reclamacao|
+---+-------------+--------------+------------------+-------------+-----------+-----------+---------------+--------------------+----------------------+----------+----------+-----

**5 - Deverá criar pelo menos duas novas colunas contendo alguma informação relevante sobre as outras colunas já existentes (Funções de Agrupamento, Agregação ou Joins).**

In [59]:
# Inserção de columa relacionamento classificação: Ótimo ou Frequente.
df1 = (df.withColumn("Relacionamento", 
       F.when(F.col("qtd_compras_online") > F.col("qtd_visitas_online_mes"), F.lit("Otimo"))
        .otherwise("Frequente")))

In [60]:
df1.show()  # Visualização na coluna Relacionamento

+---+-------------+--------------+------------------+-------------+-----------+-----------+---------------+--------------------+----------------------+----------+----------+---------+---------+----------+-----------------+------------------+------------------+--------------------+-------------------+----------------------+--------------+--------------+--------------+--------------+--------------+----------+--------------+
|   |identificacao|ano_nascimento|          educacao| estado_civil|renda_anual|qtd_crianca|qtd_adolescente|data_primeira_compra|dias_desde_ult_compras|qtd_vinhos|qtd_frutas|qtd_carne|qtd_peixe|guloseimas|qtd_produtos_gold|qtd_ofertas_compra|qtd_compras_online|qtd_compras_catalogo|qtd_compras_na_loja|qtd_visitas_online_mes|oferta_aceita1|oferta_aceita2|oferta_aceita3|oferta_aceita4|oferta_aceita5|reclamacao|Relacionamento|
+---+-------------+--------------+------------------+-------------+-----------+-----------+---------------+--------------------+--------------------

In [61]:
# Classifica clientes e visitantes em compras online
df2 = (df1.withColumn("Clientela", 
       F.when(F.col("qtd_compras_online") > 0, F.lit("Cliente"))
        .otherwise("Visitante")))

In [63]:
df2.show()  # Visualização na coluna Clientela

+---+-------------+--------------+------------------+-------------+-----------+-----------+---------------+--------------------+----------------------+----------+----------+---------+---------+----------+-----------------+------------------+------------------+--------------------+-------------------+----------------------+--------------+--------------+--------------+--------------+--------------+----------+--------------+---------+
|   |identificacao|ano_nascimento|          educacao| estado_civil|renda_anual|qtd_crianca|qtd_adolescente|data_primeira_compra|dias_desde_ult_compras|qtd_vinhos|qtd_frutas|qtd_carne|qtd_peixe|guloseimas|qtd_produtos_gold|qtd_ofertas_compra|qtd_compras_online|qtd_compras_catalogo|qtd_compras_na_loja|qtd_visitas_online_mes|oferta_aceita1|oferta_aceita2|oferta_aceita3|oferta_aceita4|oferta_aceita5|reclamacao|Relacionamento|Clientela|
+---+-------------+--------------+------------------+-------------+-----------+-----------+---------------+--------------------+

In [66]:
# Soma a quantidade de compras online e presencial do cliente
df2 = df2.withColumn("compras_total", F.col("qtd_compras_online") + F.col("qtd_compras_na_loja"))

In [67]:
df2.show()  # Visualização na coluna compras_total

+---+-------------+--------------+------------------+-------------+-----------+-----------+---------------+--------------------+----------------------+----------+----------+---------+---------+----------+-----------------+------------------+------------------+--------------------+-------------------+----------------------+--------------+--------------+--------------+--------------+--------------+----------+--------------+---------+-------------+
|   |identificacao|ano_nascimento|          educacao| estado_civil|renda_anual|qtd_crianca|qtd_adolescente|data_primeira_compra|dias_desde_ult_compras|qtd_vinhos|qtd_frutas|qtd_carne|qtd_peixe|guloseimas|qtd_produtos_gold|qtd_ofertas_compra|qtd_compras_online|qtd_compras_catalogo|qtd_compras_na_loja|qtd_visitas_online_mes|oferta_aceita1|oferta_aceita2|oferta_aceita3|oferta_aceita4|oferta_aceita5|reclamacao|Relacionamento|Clientela|compras_total|
+---+-------------+--------------+------------------+-------------+-----------+-----------+---------

In [77]:
# Dataframe grupos de clientes por educação e estado  civil
df2.groupby(F.col("educacao"), F.col("estado_civil")).count().show(50)

+------------------+-------------+-----+
|          educacao| estado_civil|count|
+------------------+-------------+-----+
|Ensino_fundamental|Uniao_estavel|   14|
|Ensino_fundamental|     Solteiro|   18|
|Ensino_fundamental|   Divorciado|    1|
|          Graduado|     Solteiro|  252|
|         Mestrando|     Solteiro|   37|
|         Mestrando|       Casado|   81|
|          Mestrado|   Divorciado|   37|
|         Mestrando|   Divorciado|   23|
|         Doutorado|Uniao_estavel|  117|
|          Graduado|       Inapto|    1|
|          Mestrado|        Viuvo|   12|
|          Mestrado|       Casado|  138|
|          Graduado|Uniao_estavel|  286|
|         Mestrando|        Viuvo|    5|
|          Mestrado| Mora_sozinho|    1|
|         Doutorado| Mora_sozinho|    1|
|          Graduado|       Casado|  433|
|          Mestrado|     Solteiro|   75|
|          Mestrado|Uniao_estavel|  106|
|         Doutorado|     Solteiro|   98|
|Ensino_fundamental|        Viuvo|    1|
|          Mestr

**6 - Deverá utilizar filtros, ordenação e agrupamento, trazendo dados relevantes para o negócio em questão.**

In [78]:
# Tabela com infomações de clientes que reclamaram
df2.select(F.col("identificacao"), F.col("educacao"), F.col("estado_civil"), F.col("compras_total")).filter(F.col("reclamacao") == "sim").show()

+-------------+---------+-------------+-------------+
|identificacao| educacao| estado_civil|compras_total|
+-------------+---------+-------------+-------------+
|        10401|Mestrando|Uniao_estavel|            3|
|         3120| Graduado|Uniao_estavel|            3|
|         7829|Mestrando|   Divorciado|            4|
|         5726| Mestrado|     Solteiro|            7|
|         6201| Graduado|     Solteiro|            5|
|         6142| Mestrado|       Casado|           10|
|         3710|Doutorado|     Solteiro|            4|
|        10995|Mestrando|       Casado|            7|
|         6730| Graduado|       Casado|            5|
|         2875| Graduado|       Casado|           16|
|          359| Graduado|Uniao_estavel|           14|
|         9500| Graduado|       Casado|           17|
|         7192| Graduado|Uniao_estavel|           12|
|        10637| Graduado|       Casado|            6|
|        10971| Graduado|     Solteiro|           10|
|         9483| Graduado|Uni

In [79]:
# Conta numeros de reclamações

df2.select(F.col("identificacao"), F.col("educacao"), F.col("estado_civil"), F.col("compras_total")).filter(F.col("reclamacao") == "sim").count()  

21

In [80]:
# Quantidade de Clientes que não possuem filhos: Crianças ou adolecentes

filtro1 = F.col("qtd_crianca") == 0
filtro2 = F.col("qtd_adolescente") == 0

df2.select(F.col("qtd_crianca"),
           F.col("qtd_adolescente")).filter(filtro1 & filtro2).count()

638

**7 - Utilizar pelo menos duas Window Functions.**

In [81]:
df2.select(F.col("identificacao"),
           F.col("dias_desde_ult_compras"), F.col("renda_anual")).orderBy(F.col("dias_desde_ult_compras").desc()).show()

+-------------+----------------------+-----------+
|identificacao|dias_desde_ult_compras|renda_anual|
+-------------+----------------------+-----------+
|          868|                    99|    44794.0|
|         7829|                    99|    36640.0|
|         7232|                    99|    42429.0|
|        10142|                    99|    66476.0|
|         9977|                    99|    78901.0|
|         8595|                    99|    42429.0|
|         7947|                    99|    42231.0|
|         5263|                    99|    31056.0|
|         3363|                    99|    20130.0|
|         4070|                    99|    94871.0|
|         2831|                    99|    78416.0|
|         2415|                    99|    62568.0|
|         1743|                    99|    69719.0|
|         7212|                    99|    44794.0|
|         2106|                    99|    20130.0|
|          528|                    99|    65819.0|
|           22|                

In [82]:
w0 = Window.partitionBy(F.col("ano_nascimento")).orderBy("renda_anual")
df2.withColumn("valor_referencia", F.row_number().over(w0)).show()

+----+-------------+--------------+---------+-------------+-----------+-----------+---------------+--------------------+----------------------+----------+----------+---------+---------+----------+-----------------+------------------+------------------+--------------------+-------------------+----------------------+--------------+--------------+--------------+--------------+--------------+----------+--------------+---------+-------------+----------------+
|    |identificacao|ano_nascimento| educacao| estado_civil|renda_anual|qtd_crianca|qtd_adolescente|data_primeira_compra|dias_desde_ult_compras|qtd_vinhos|qtd_frutas|qtd_carne|qtd_peixe|guloseimas|qtd_produtos_gold|qtd_ofertas_compra|qtd_compras_online|qtd_compras_catalogo|qtd_compras_na_loja|qtd_visitas_online_mes|oferta_aceita1|oferta_aceita2|oferta_aceita3|oferta_aceita4|oferta_aceita5|reclamacao|Relacionamento|Clientela|compras_total|valor_referencia|
+----+-------------+--------------+---------+-------------+-----------+-----------

In [83]:
df2.withColumn("percent_rank",percent_rank().over(w0)).show()

+----+-------------+--------------+---------+-------------+-----------+-----------+---------------+--------------------+----------------------+----------+----------+---------+---------+----------+-----------------+------------------+------------------+--------------------+-------------------+----------------------+--------------+--------------+--------------+--------------+--------------+----------+--------------+---------+-------------+-------------------+
|    |identificacao|ano_nascimento| educacao| estado_civil|renda_anual|qtd_crianca|qtd_adolescente|data_primeira_compra|dias_desde_ult_compras|qtd_vinhos|qtd_frutas|qtd_carne|qtd_peixe|guloseimas|qtd_produtos_gold|qtd_ofertas_compra|qtd_compras_online|qtd_compras_catalogo|qtd_compras_na_loja|qtd_visitas_online_mes|oferta_aceita1|oferta_aceita2|oferta_aceita3|oferta_aceita4|oferta_aceita5|reclamacao|Relacionamento|Clientela|compras_total|       percent_rank|
+----+-------------+--------------+---------+-------------+-----------+-----

### **Carregamento no Drive**

In [98]:
# Salva local 
df2.write.csv("df_spark_marketing_campaign.csv")

In [101]:
# Salva local na pasta "/content/"
df2.write.format("com.databricks.spark.csv").option("header", "true").option("inferschema", "true")\
    .option("delimiter", ";").save("/content/df_spark_marketing_campaign.csv")

### **Carregamento no GCP**

In [93]:
# Salva no GCP
df2.coalesce(1).write.mode('overwrite').csv(path)